# **This code is an implementation of a chatbot using the GPT-2 language model from Hugging Face's Transformers library.**
<br>
Reference: https://huggingface.co/transformers/v2.2.0/pretrained_models.html



In [1]:
# @title
!pip install transformers[torch] -U
!pip install torch
!pip install -U PyPDF2
!pip install python-docx



In [2]:
import os
import re
from PyPDF2 import PdfReader
import docx
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments


Required functions to read text from various files located in a directory. Files can be a mix of pdf, docx, or txt.

In [3]:

# Functions to read different file types
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
        elif filename.endswith(".docx"):
            combined_text += read_word(file_path)
        elif filename.endswith(".txt"):
            combined_text += read_txt(file_path)
    return combined_text



The train_chatbot function uses the combined text data to train a GPT-2 model using the provided training arguments. The resulting trained model and tokenizer are then saved to a specified output directory.

In [4]:
def train_chatbot(directory, model_output_path, train_fraction=0.8):
    # Read documents from the directory
    combined_text = read_documents_from_directory(directory)
    combined_text = re.sub(r'\n+', '\n', combined_text).strip()  # Remove excess newline characters

    # Split the text into training and validation sets
    split_index = int(train_fraction * len(combined_text))
    train_text = combined_text[:split_index]
    val_text = combined_text[split_index:]

    # Save the training and validation data as text files
    with open("train.txt", "w") as f:
        f.write(train_text)
    with open("val.txt", "w") as f:
        f.write(val_text)

    # Set up the tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl
    model = GPT2LMHeadModel.from_pretrained("gpt2-large")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl

    # Prepare the dataset
    train_dataset = TextDataset(tokenizer=tokenizer, file_path="train.txt", block_size=128)
    val_dataset = TextDataset(tokenizer=tokenizer, file_path="val.txt", block_size=128)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Set up the training arguments
    training_args = TrainingArguments(
        output_dir=model_output_path,
        overwrite_output_dir=True,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=100,
        save_steps=10_000,
        save_total_limit=2,
        logging_dir='./logs',
    )

    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()
    trainer.save_model(model_output_path)

    # Save the tokenizer
    tokenizer.save_pretrained(model_output_path)


The generate_response function takes a trained model, tokenizer, and a prompt string as input and generates a response using the GPT-2 model.

In [5]:
def generate_response(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


The main function is the entry point for the program. It specifies the path to the directory containing the training data and the path to the output directory for the trained model and tokenizer. It then trains the chatbot using the train_chatbot function and generates a response to a specified prompt using the generate_response function.

In [8]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def train_chatbot(directory, model_output_path):
    # Add your training code here
    # For now, let's just save an empty model and tokenizer to the path
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model.save_pretrained(model_output_path)
    tokenizer.save_pretrained(model_output_path)

def generate_response(model, tokenizer, prompt):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=100, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

def main():
    # Replace with the path to your directory containing the files
    directory = "C:\\Users\\Pasindu\\chatNew\\training_data"  
    # Replace with the path to your model output directory
    model_output_path = "C:\\Users\\Pasindu\\chatNew\\model_output"

    # Train the chatbot
    train_chatbot(directory, model_output_path)

    # Load the fine-tuned model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_output_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)


if __name__ == "__main__":
    main()


## Now, let us test the model.
<p>
Use the following code if you are only performing inference (generating text). This can be placed in a separate notebook.

In [9]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


In [10]:
def generate_response(model, tokenizer, prompt, max_length=250):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [11]:
model_path = "C:\\Users\\Pasindu\\chatNew\\model_output"
# Load the fine-tuned model and tokenizer
my_chat_model = GPT2LMHeadModel.from_pretrained(model_path)
my_chat_tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In the case of the GPT-2 tokenizer, the model uses a byte-pair encoding (BPE) algorithm, which tokenizes text into subword units. As a result, one word might be represented by multiple tokens.

For example, if you set max_length to 50, the generated response will be limited to 50 tokens, which could be fewer than 50 words, depending on the text.

In [12]:
prompt = "What is the mitosis?"
response = generate_response(my_chat_model, my_chat_tokenizer, prompt, max_length=150)
print("Generated response:", response)


Generated response: What is the mitosis?

The mitosis is a process of the body's immune system reacting to a specific antigen. The immune system is able to recognize the antigen and recognize it as a specific antigen. The immune system can then recognize the antigen and recognize it as a specific antigen.

The mitosis is a process of the body's immune system reacting to a specific antigen. The immune system is able to recognize the antigen and recognize it as a specific antigen. The mitosis is a process of the body's immune system reacting to a specific antigen.

The mitosis is a process of the body's immune system reacting to a specific antigen.

The mitosis is a process of the body's immune system reacting to
